In [1]:
import numpy as np
import pandas as pd
from starfish.plot import *

In [2]:
from starfish.set import settings
settings.cosmo_value

[0.32, 0.049, 0.67, 0.96, 0.81, 0.06, 3.044]

In [3]:
def var_filter(fisher,lcdm,neff_fixed):
    df=fisher.copy()
    if lcdm:
        df=df.drop('w0',axis=0)
        df=df.drop('w0',axis=1)
        df=df.drop('wa',axis=0)
        df=df.drop('wa',axis=1)
    if neff_fixed:
        df=df.drop('N_eff',axis=0)
        df=df.drop('N_eff',axis=1)

    return df

# nuisance parameters of different tracers/surveys should be independent of each other

In [4]:
DESI_multi=var_filter(pd.read_excel('../data/DESI_all/valid.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=False)
cov=pd.DataFrame(np.linalg.inv(DESI_multi.values),index=DESI_multi.index,columns=DESI_multi.columns)
DESI_multi_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

# CMB-S4 + LiteBIRD + DESI multitracer

In [5]:
cmb_fisher=var_filter(pd.read_excel('../data/CMB/all.xlsx',header=0,index_col=0),lcdm=True,neff_fixed=False)
cmb_fisher

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,tau
omega_m0,1.851288e+08,-6.928981e+08,1.371057e+08,-4.740936e+06,-5.782434e+07,-1.244715e+07,-6.589565e+06,4.485034e+07
omega_b0,-6.928981e+08,3.023402e+09,-4.583542e+08,3.206098e+07,2.226094e+08,4.714503e+07,2.328230e+07,-1.714219e+08
h,1.371057e+08,-4.583542e+08,1.125702e+08,-1.894718e+06,-3.817385e+07,-7.840066e+06,-5.484485e+06,2.962660e+07
n_s,-4.740936e+06,3.206098e+07,-1.894718e+06,1.046895e+06,1.665063e+06,3.149959e+05,1.125701e+05,-1.284039e+06
sigma_8,-5.782434e+07,2.226094e+08,-3.817385e+07,1.665063e+06,2.236167e+07,5.252308e+06,1.591679e+06,-1.731189e+07
m_nu,-1.244715e+07,4.714503e+07,-7.840066e+06,3.149959e+05,5.252308e+06,1.282975e+06,2.946820e+05,-4.090356e+06
N_eff,-6.589565e+06,2.328230e+07,-5.484485e+06,1.125701e+05,1.591679e+06,2.946820e+05,2.884944e+05,-1.230550e+06
tau,4.485034e+07,-1.714219e+08,2.962660e+07,-1.284039e+06,-1.731189e+07,-4.090356e+06,-1.230550e+06,1.374361e+07


In [6]:
cmb_cov=pd.DataFrame(np.linalg.inv(cmb_fisher.values),index=cmb_fisher.index,columns=cmb_fisher.columns)
cmb_cov_marg=cmb_cov.drop('tau',axis=0)
cmb_cov_marg=cmb_cov_marg.drop('tau',axis=1)
cmb_fisher_marg=pd.DataFrame(np.linalg.inv(cmb_cov_marg),index=settings.var_name,columns=settings.var_name)
cmb_fisher_marg

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff
omega_m0,3.876607e+07,-1.334868e+08,4.042346e+07,-5.506558e+05,-1.329430e+06,9.011528e+05,-2.573839e+06
omega_b0,-1.334868e+08,8.852823e+08,-8.882606e+07,1.604536e+07,6.680925e+06,-3.873349e+06,7.933847e+06
h,4.042346e+07,-8.882606e+07,4.870520e+07,8.732384e+05,-8.552429e+05,9.773665e+05,-2.831834e+06
n_s,-5.506558e+05,1.604536e+07,8.732384e+05,9.269296e+05,4.764654e+04,-6.715809e+04,-2.397812e+03
sigma_8,-1.329430e+06,6.680925e+06,-8.552429e+05,4.764654e+04,5.550676e+05,9.996606e+04,4.163926e+04
m_nu,9.011528e+05,-3.873349e+06,9.773665e+05,-6.715809e+04,9.996606e+04,6.560846e+04,-7.155261e+04
N_eff,-2.573839e+06,7.933847e+06,-2.831834e+06,-2.397812e+03,4.163926e+04,-7.155261e+04,1.783157e+05


In [7]:
fisher1_combined=DESI_multi_valid+cmb_fisher_marg
fisher1_combined.to_excel('../data/combined/cmb+dmul-lcdm-free.xlsx')
cov1_combined=pd.DataFrame(np.linalg.inv(fisher1_combined),index=fisher1_combined.index,columns=fisher1_combined.columns)
cov1_combined

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff
omega_m0,3.769776e-06,2.849769e-07,-1.622460e-06,2.108671e-07,-2.012439e-06,0.000019,0.000024
omega_b0,2.849769e-07,4.795590e-08,-3.342323e-07,-2.098090e-07,-4.742309e-07,0.000002,-0.000003
h,-1.622460e-06,-3.342323e-07,2.704199e-06,1.477417e-06,3.576691e-06,-0.000009,0.000030
n_s,2.108671e-07,-2.098090e-07,1.477417e-06,3.767027e-06,1.208092e-06,0.000007,0.000039
sigma_8,-2.012439e-06,-4.742309e-07,3.576691e-06,1.208092e-06,9.115277e-06,-0.000028,0.000036
m_nu,1.943316e-05,1.602022e-06,-8.976760e-06,7.188830e-06,-2.762242e-05,0.000194,0.000151
N_eff,2.422715e-05,-2.587602e-06,3.005953e-05,3.858440e-05,3.578529e-05,0.000151,0.001002


In [8]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov1_combined.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 1.9e-03
-- omega_b0: 2.2e-04
--        h: 1.6e-03
--      n_s: 1.9e-03
--  sigma_8: 3.0e-03
--     m_nu: 1.4e-02
--    N_eff: 3.2e-02
